In [54]:
from striprtf.striprtf import rtf_to_text
from dotenv import load_dotenv
import pandas as pd
import openai
import json

In [45]:
load_dotenv()

True

In [46]:
with open("search-directory.rtf") as infile:
    content = infile.read()
    text = rtf_to_text(content)

In [47]:
splitted_text = list(map(lambda string: string.strip(), text.split("------")))

In [48]:
print(len(splitted_text))

101


In [49]:
def generate_prompt(text):
    return f"""
    Extract the
    first name as 'first_name',
    last name as 'last_name',
    SIGs as 'sigs',
    job title as 'job_title',
    company as 'company',
    chapter as 'chapter',
    work phone as 'phone',
    work street as 'street',
    work city as 'city',
    work state as 'state',
    work zip as 'zip',
    primary email as 'primary_email',
    secondary email as 'secondary_email',
    and LinkedIn address as 'linkedin_address'
    in JSON format.
    If there is no information for any given field, the value should be an empty string.
    The data is the following: {text}
"""

In [50]:
results = []

In [51]:
for text in splitted_text:
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that will help extraction information from text in JSON format."},
                {"role": "user", "content": generate_prompt(text)},
            ]
        )
        json_response = response["choices"][0]["message"]["content"]
        results.append(json.loads(json_response))
    except Exception:
        pass

In [53]:
len(results)

97

In [55]:
df = pd.DataFrame(results)

In [56]:
df

,first_name,last_name,sigs,job_title,company,chapter,phone,street,city,state,zip,primary_email,secondary_email,linkedin_address
0,Jim,Aamoth,Banking; Information Technology; Manufacturing,Chief Financial Officer,Oneil Digital Solutions,"Los Angeles (Hollywood/Santa Monica), CA",(818) 723-8668,12655 Beatrice Street,Los Angeles,CA,90066,Jimaam@cs.com,,
1,Christian,Abadiotakis,Asset Management,Chief Financial Officer,Dreman Value Management,"Bergen County, NJ; New York, NY; Warren, NJ",,227 Nottingham Road,Morganville,NJ,07751,cabadiotakis@gmail.com,cabadiotakis@dreman.com,
2,Ziad,Abanni,Energy & Natural Resources; High Technology; I...,Chief Financial Officer,United States,"Abu Dhabi/Dubai, UAE",,"Lebanon, Aley, Dubai, 3033, United Arab Emirates",,,,z_abanni@hotmail.com,,
3,Martin,Abbene,Capital Markets; Food Industry; Manufacturing;...,Chief Financial Officer,"Altis Aero Systems, Inc.","Atlanta, GA",(912) 748-1800 ext: 305,101 Coleman Boulevard,Savannah,GA,31408,mjabbene@bellsouth.net,mabbene@altisaero.com,
4,Bradley,Abbott,Capital Markets; Healthcare; Investor Relation...,Chief Financial Officer & Treasurer,"Clarke Power Services, Inc.","Cincinnati, OH",(513) 719-2397,3133 East Kemper Road,Cincinnati,OH,45241,brad.abbott@mac.com,theabbotts@mac.com,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,George,Allen,Healthcare; Information Technology; Internatio...,Chief Financial Officer,United States,"Dallas, TX",(214) 732-4464,1217 Middle Cove Drive,Plano,TX,75023,gsallen3@yahoo.com,,
93,James,Allen,Not-for-Profit,Chief Financial Officer,,"Houston, TX; Houston (Katy/Sugar Land), TX; Ho...",(281) 304-9833,14315 Spyglen Lane,Cypress,TX,77429,jamesallen163@hotmail.com,jim.allen@boysandgirlscountry.org,
94,Mark,Allen,Capital Markets; High Technology; Investor Rel...,Chief Financial Officer,,"Baltimore, MD; Boston, MA; Orlando, FL; Norfol...",(386) 299-5906,"4670 Links Village Drive, Unit C-202",Ponce Inlet,FL,32127,mgamga2003@yahoo.com,mark.allen@concurrentmfg.com,https://www.linkedin.com/in/mark-allen-097b42b/
95,Michael,Allen,Food Industry; Manufacturing; Retailing; Aeros...,Chief Financial Officer,United States,"Westport, CT",,51 Overlook Drive,New Canaan,CT,06840,mdallen6@aol.com,mdallen2844@gmail.com,


In [60]:
df.to_excel("contacts.xlsx", index=False)